<a href="https://colab.research.google.com/github/rgolds5/DS-Unit-2-Linear-Models/blob/master/Module_2_Doing_Linear_Regression_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

model = LinearRegression()
encoder = ce.OneHotEncoder(use_cat_names = True)
scaler = StandardScaler()

data_url = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/nyc/nyc-rent-2016.csv'

df = pd.read_csv(data_url)
assert df.shape == (48300, 34)



In [6]:
df.created = pd.to_datetime(df.created, infer_datetime_format = True)
df.month = df.created.dt.month

train = df[df.month < 6]
test = df[df.month == 6]

train.price.mean()

3432.7534190068222

In [15]:
features = [
    'bathrooms',
    'bedrooms',
    'longitude',
    ]

target = 'price'

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mean_absolute_error(y_test, y_pred)

667.293314729857

In [50]:
features = [
    'bathrooms',
    'bedrooms',
    'longitude',
    'interest_level',
    'doorman',
    ]

target = 'price'

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'Intercept: {model.intercept_}')
model.coef_
              

MAE: 606.3687175807665
Intercept: 3433.054238785847


array([ 5.41475931e+02,  5.58567119e+02, -3.73131401e+02,  2.47282022e+15,
        4.24591758e+15,  3.87714519e+15,  2.38065790e+02])

I ran the above code with multiple sets of feature combinations. The five features I have finished with, _bathrooms_, _bedrooms_, _longitude_, _interest_level_, and _doorman_ seemed to be the ones that gave me the most _bang for my buck_.